In [ ]:
import os
import argparse
import time
!pip install mxnet
!pip install gluoncv

In [ ]:
import numpy as np
import mxnet as mx
from mxnet.gluon.data.vision import transforms
import gluoncv
# using cpu
ctx = mx.cpu(0)

In [ ]:
url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTqnj9rI5cYgXD9tg8s3SOlet16xMeFdz59AA&usqp=CAU'
filename = 'test_img.png'
gluoncv.utils.download(url, filename, True)
import PIL.Image as pil
img = pil.open(filename).convert('RGB')
from matplotlib import pyplot as plt
plt.imshow(img)
plt.show()

In [ ]:
original_width, original_height = img.size
feed_height = 192
feed_width = 640
img = img.resize((feed_width, feed_height), pil.LANCZOS)
img = transforms.ToTensor()(mx.nd.array(img)).expand_dims(0).as_in_context(context=ctx)
model = gluoncv.model_zoo.get_model('monodepth2_resnet18_kitti_stereo_640x192',pretrained_base=False, ctx=ctx, pretrained=True)
outputs = model.predict(img)
disp = outputs[("disp", 0)]
disp_resized = mx.nd.contrib.BilinearResize2D(disp, height=original_height, width=original_width)


In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm
disp_resized_np = disp_resized.squeeze().as_in_context(mx.cpu()).asnumpy()
vmax = np.percentile(disp_resized_np, 95)
normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
im = pil.fromarray(colormapped_im)
im.save('test_output.png')

import matplotlib.image as mpimg
disp_map = mpimg.imread('test_output.png')
plt.imshow(disp_map)
plt.show()